My very first project with supervised deep learning. I have been reading "Deep Learning with Python" by françois chollet. I am not finished with it, but I really wanted to create my own solution on a given data set. 

The dataset I will be using is https://www.kaggle.com/sonalidasgupta95/churn-prediction-of-bank-customers/downloads/churn-prediction-of-bank-customers.zip/1
It will be a binary classification problem to determine if a customer will exit the bank.

In [3]:
import keras
import pandas as pd

data = pd.read_csv('Churn_Modelling.csv')

data.head()

Using TensorFlow backend.


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


I am going to use CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, and EstimatedSalary to determine whether the customer will exit. 
I will preprocess the data as follows:
    Geography: France will be 0, Spain will be 1, and Germany will be 2
    Gender: Female will be 0, Male will be 1
    Balance: Get the mean then divide by the standard deviation
    EstimatedSalary: Get the mean then divide by the standard deviation

In [4]:
import csv
import numpy as np

# Getting Data
with open('Churn_Modelling.csv', 'r') as f:
    data = list(csv.reader(f, delimiter=';'))

for i in range(len(data)):
    data[i] = data[i][0].split(',')
    
    # Switching gender to numeric
    if data[i][5] == "Female":
        data[i][5] = 0
    elif data[i][5] == "Male":
        data[i][5] = 1
    
    # Switching geography to numeric
    if data[i][4] == "France":
        data[i][4] = 0
    elif data[i][4] == "Spain":
        data[i][4] = 1
    elif data[i][4] == "Germany":
        data[i][4] = 2
    
    data[i].pop(2)
    data[i].pop(1)
    data[i].pop(0)
    
    if i != 0:
        for j in range(len(data[i])):
            data[i][j] = float(data[i][j])
            
print(data[0])
data.pop(0)
data = np.array(data, np.float64)
print(data[0])

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']
[6.1900000e+02 0.0000000e+00 0.0000000e+00 4.2000000e+01 2.0000000e+00
 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0134888e+05
 1.0000000e+00]


In [5]:
import math

# Splitting the data into train and test
np.random.shuffle(data)

# 75% of the data will be training data
numTrain = math.floor(len(data) * .75)

trainData = data[:numTrain]
testData = data[numTrain:]

trainTarget = []
testTarget = []

for i in range(len(trainData)):
    trainTarget.append(trainData[i][10])
for i in range(len(testData)):
    testTarget.append(testData[i][10])
    
trainData = np.delete(trainData, 10, 1)
testData = np.delete(testData, 10, 1)

Splitting data into two: train data and test data. Also parsing out the target for learning purposes.
Now I will normalize the data by subtracting the mean off a data set and dividing it by the standard deviation.
This will get me numbers that are closer to each other.

In [6]:
# Normalization
credit = []
salary = []
balance = []

for i in range(len(trainData)):
    credit.append(trainData[i][0])
    balance.append(trainData[i][5])
    salary.append(trainData[i][9])

creditMean = np.mean(credit)
salaryMean = np.mean(salary)
balanceMean = np.mean(balance)
creditStd = np.std(credit)
salaryStd = np.std(salary)
balanceStd = np.std(balance)

trainData[:, 0] -= creditMean
trainData[:, 0] /= creditStd
trainData[:, 5] -= balanceMean
trainData[:, 5] /= balanceStd
trainData[:, 9] -= salaryMean
trainData[:, 9] /= salaryStd

print(trainData[:5, 0])
print(trainData[:5, 5])
print(trainData[:5, 9])

[-0.19942426  0.52938947 -0.53259625  0.87297308 -1.72993309]
[ 0.50608503 -1.22388516 -0.39134859 -0.08152065 -1.22388516]
[-0.8225009  -0.90239685 -0.25546344 -1.05673918  1.31325501]


Credit score, salary, and balance are now all normalized and ready to be inputted into our model.

In [19]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(trainData, trainTarget, epochs=20, batch_size=512, validation_data=(testData, testTarget))

Train on 7500 samples, validate on 2500 samples
Epoch 1/20
7500/7500 [==============================] - 1s 110us/step - loss: 0.7721 - acc: 0.7316 - val_loss: 3.6646 - val_acc: 0.7720
Epoch 2/20
7500/7500 [==============================] - ETA: 0s - loss: 0.5539 - acc: 0.781 - 0s 12us/step - loss: 0.5431 - acc: 0.7843 - val_loss: 3.7011 - val_acc: 0.7692
Epoch 3/20
7500/7500 [==============================] - 0s 13us/step - loss: 0.5673 - acc: 0.7725 - val_loss: 3.8571 - val_acc: 0.7600
Epoch 4/20
7500/7500 [==============================] - 0s 10us/step - loss: 0.5284 - acc: 0.7865 - val_loss: 3.7370 - val_acc: 0.7676
Epoch 5/20
7500/7500 [==============================] - 0s 16us/step - loss: 0.5405 - acc: 0.7721 - val_loss: 3.9145 - val_acc: 0.7564
Epoch 6/20
7500/7500 [==============================] - 0s 13us/step - loss: 0.5259 - acc: 0.7847 - val_loss: 3.9312 - val_acc: 0.7552
Epoch 7/20
7500/7500 [==============================] - 0s 12us/step - loss: 0.5286 - acc: 0.7831 - val